In [28]:
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
import numpy as np
import re

# CSV Data Dec-Jan 2022-2023

In [3]:
df_All = pd.read_csv('./DataWaypoints/JTOSJW_dec_jan_2022.csv') 

# Format the coordinates to fit future handling
df_All['Latitude'] = df_All['Latitude'] / 1000000
df_All['Longitude'] = df_All['Longitude'] /1000000

df_All #571087 rows


,JId,CreatedOn,SearchStart,SearchEnd,StartStop,EndStop,StartZone,Endzone,internalStartZones,internalValidZones,SJId,Id,Name,Latitude,Longitude,Type,SJSearchJourney_Id
0,b75f6ffa-4aa0-48ba-87d5-4b5bb5ff289f,2022-12-04 10.37.35.3187198,Enghave Plads St. (Metro) (01),Enghave Plads St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",2bb3d850-8e8f-4727-a011-c67f314b12b5,8603344.0,Enghave Plads St. (Metro),55.667283,12.545813,M,11ba19ec-1f6c-4b0b-a563-0003e7eb3b81
1,b75f6ffa-4aa0-48ba-87d5-4b5bb5ff289f,2022-12-04 10.37.35.3187198,Enghave Plads St. (Metro) (01),Enghave Plads St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",0afe8865-2e99-4b4d-9dce-dcebf657f517,8600856.0,Ørestad St.,55.628432,12.578776,waypoint,11ba19ec-1f6c-4b0b-a563-0003e7eb3b81
2,b75f6ffa-4aa0-48ba-87d5-4b5bb5ff289f,2022-12-04 10.37.35.3187198,Enghave Plads St. (Metro) (01),Enghave Plads St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",17597387-c263-45ec-ad33-eac9fdbaf309,8603330.0,København H (Metro),55.671940,12.564124,waypoint,11ba19ec-1f6c-4b0b-a563-0003e7eb3b81
3,b75f6ffa-4aa0-48ba-87d5-4b5bb5ff289f,2022-12-04 10.37.35.3187198,Enghave Plads St. (Metro) (01),Enghave Plads St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",d82ecfdc-3804-4b7b-9ad9-3535c8970881,NaN,København H (Metro),55.671940,12.564124,WALK,11ba19ec-1f6c-4b0b-a563-0003e7eb3b81
4,b75f6ffa-4aa0-48ba-87d5-4b5bb5ff289f,2022-12-04 10.37.35.3187198,Enghave Plads St. (Metro) (01),Enghave Plads St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",7a6d9cdc-97f1-40f9-9c5a-23c4e71c1a3d,8600626.0,København H,55.673063,12.565562,REG,11ba19ec-1f6c-4b0b-a563-0003e7eb3b81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571082,c6c11ad9-ebad-4d66-8385-cdf81e3a6294,2022-12-03 08.08.08.8184281,Orientkaj St. (Metro) (01),Orientkaj St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",c252e8fa-b96f-45bd-a1a1-1344f3fa3099,8600650.0,Østerport St.,55.692498,12.587784,IC,8fae5c1a-0ce7-4682-9112-ffffed47caae
571083,c6c11ad9-ebad-4d66-8385-cdf81e3a6294,2022-12-03 08.08.08.8184281,Orientkaj St. (Metro) (01),Orientkaj St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",9520e9d8-0af6-4836-90bb-016fc9234ab8,8603345.0,Orientkaj St. (Metro),55.711609,12.595272,M,8fae5c1a-0ce7-4682-9112-ffffed47caae
571084,c6c11ad9-ebad-4d66-8385-cdf81e3a6294,2022-12-03 08.08.08.8184281,Orientkaj St. (Metro) (01),Orientkaj St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",61032bbb-fe40-4062-92ae-256ec02f9570,8600646.0,Nørreport St.,55.683455,12.571801,waypoint,8fae5c1a-0ce7-4682-9112-ffffed47caae
571085,c6c11ad9-ebad-4d66-8385-cdf81e3a6294,2022-12-03 08.08.08.8184281,Orientkaj St. (Metro) (01),Orientkaj St. (Metro) (01),NaN,NaN,0,0,1001,"1001,1002,1003",8280fdac-4469-4624-8523-4d383718bb04,NaN,Østerport St.,55.692498,12.587784,waypoint,8fae5c1a-0ce7-4682-9112-ffffed47caae


## Data analysis

This dataset is retrieved using sql-query: 

```SQL
SELECT J.Id as JId, J.CreatedOn, J.SearchStart, J.SearchEnd, J.StartStop, J.EndStop,
J.StartZone, J.Endzone, J.internalStartZones, 
J.internalValidZones, SJWaypoints._id as SJId, SJWaypoints.Id,  
SJWaypoints.Name, SJWaypoints.Latitude, 
SJWaypoints.Longitude, SJWaypoints.[Type], SJWaypoints.SJSearchJourney_Id
FROM Journeys J
JOIN Tickets ON J.Id = Tickets.Journey_Id
JOIN Orders ON Orders.Id = Tickets.OrderId
JOIN SJSearchJourneys SJ ON SJ.Id = Orders.JourneyClasses_Id
JOIN SJWaypoints ON SJWaypoints.SJSearchJourney_Id = SJ.Id
WHERE J.CreatedOn BETWEEN '2022-12-01 00:00:00' and '2023-01-01 00:00:00'
```

The idea is that we have now joined Journeys with, Tickets, Orders, SJSearchJourneys and lastly SJWaypoints. The results should, from the ER digram given by the contact person of the database. 

So now we should be left with all journeys that are travelled where the ticket has been bought using the DOT-Billet app. 

Firstly: We are only interested in journeys made in Copenhagen, so therefor we want to filter out all journeys that are not taken in the zones 1001-1004.

In [4]:

# First condition, check if startZone and endZone match within the limits
condition_1 = (
    (df_All['StartZone'].between(1, 4) | df_All['StartZone'].between(1001, 1004))
     & 
     (df_All['Endzone'].between(1, 4) | df_All['Endzone'].between(1001, 1004))
     )

# Second condition need to handle SectionStartZones. Here we use regex to match the string pattern in the column. We want to see if the string only contain zones inside Cph.
# We say 'The string must begin with either 1001, 1002, 1003 or 1004. Seperated by a comma, either 0 or more iterations of the one of the 4 numbers may follow the initial number'.
condition_2 = (df_All['internalStartZones'].str.match(r'^(1001|1002|1003|1004)(,(1001|1002|1003|1004))*$'))

# Third condition. Similar approach as to the second condition
condition_3 = (df_All['internalValidZones'].str.match(r'^(1001|1002|1003|1004)(,(1001|1002|1003|1004))*$'))



Secondly we want to make sequences from the stations in the journeys that has one of the three properties: 
- Is the startstop
- Is the endstop
- Is a transitioning stop


Making a new dataframe consisting of the stops that adheres to the above 3 conditions and therefor are journeys travlled in Copenhagen

In [5]:

#df_cph_Journeys_check = df_All[(condition_1 | condition_2 | condition_3)].groupby('SJSearchJourney_Id').agg(list)
df_cph_Journeys_check = df_All[(condition_1 | condition_2 | condition_3)]


# check if there are any rows where both SearchStart and SearchEnd are NaN
rows_with_startstop_endstop_nan = df_cph_Journeys_check[(df_cph_Journeys_check['StartStop'].isnull()) & (df_cph_Journeys_check['EndStop'].isnull())]
rows_with_nan_start_end = df_cph_Journeys_check[(df_cph_Journeys_check['SearchStart'].isnull()) & (df_cph_Journeys_check['SearchEnd'].isnull())]
rows_with_nan_searchStart_not_SearchEnd = df_cph_Journeys_check[(df_cph_Journeys_check['SearchStart'].isnull())]
rows_with_nan_searchEnd_not_SearchStart = df_cph_Journeys_check[(df_cph_Journeys_check['SearchEnd'].isnull())]
rows_where_all_are_nan = df_cph_Journeys_check[(df_cph_Journeys_check['StartStop'].isnull()) & (df_cph_Journeys_check['EndStop'].isnull()) & (df_cph_Journeys_check['SearchStart'].isnull()) & (df_cph_Journeys_check['SearchEnd'].isnull())]
rows_where_all_are_nan = df_cph_Journeys_check[(df_cph_Journeys_check['StartStop'].isnull()) & (df_cph_Journeys_check['EndStop'].isnull()) & (df_cph_Journeys_check['SearchStart'].isnull()) & (df_cph_Journeys_check['SearchEnd'].isnull())]
# print(rows_with_nan_start_end) #There are 17840 rows where SearchStart AND SearchEnd are NaN
# print(rows_with_startstop_endstop_nan) #There are 178000 rows where StartStop AND EndStop are NaN
# print(rows_where_all_are_nan) #There are 5706 where all four are NaN or Null
#print(rows_with_nan_searchStart_not_SearchEnd) #17840
#print(rows_with_nan_searchEnd_not_SearchStart) #17934 there are 94 journeys where searchEnd is empty


## Making sequences

***STEP 1. Grouping by the searchJourneyId's***

In [6]:

df_cph_Journeys = df_All[(condition_1 | condition_2 | condition_3)].groupby('SJSearchJourney_Id')
df_cph_Journeys
#Checking for NULL or NaN names
#num_null_names = df_cph_Journeys_check['Name'].isnull().sum() #0 

***STEP 2. Defining a function that identifies the startStop waypoint, the endStop waypoint and all transitioning waypoints***

In [7]:
def remove_non_alphanumeric(s):
        if isinstance(s, str):
            return re.sub(r'[^a-zæøåA-ZÆØÅ\s]', '', s)
        else:
            return s

Second try:

In [8]:
from difflib import SequenceMatcher


def create_sequence(group):
    # Remove non-alphanumeric characters from the SearchStart and SearchEnd columns
    group['SearchStart'] = group['SearchStart'].apply(remove_non_alphanumeric)
    group['SearchEnd'] = group['SearchEnd'].apply(remove_non_alphanumeric)

     # Find the starting waypoint
    start_waypoint = group['SearchStart'].iloc[0] if not pd.isnull(group['SearchStart'].iloc[0]) else None
    
     #In this collection, we might not be able to find the last stop - the destination of a journey.
     #if SearchEnd is not null, look through the list of names for the most similar one
    if not pd.isna(group['SearchEnd'].iloc[0]):
        names = group['Name'].values.tolist()
        end = group['SearchEnd'].iloc[0]
        
        #Find the station in names that is most similar to the searchEnd station 
        # To "take" it out of the list
        currhigh = ('', 0)
        for name in names:
            s = SequenceMatcher(a=end, b=name).ratio()
            if s > currhigh[1]:
                currhigh = (name, s)
        end_waypoint = currhigh[0] 
    else: end_waypoint = None
   
    # Find the intermediate waypoints
    intermediate_waypoints = group.loc[(group['Type'] != 'WALK') & (group['Type'] != 'waypoint') & (group['Name'] != end_waypoint), 'Name'].tolist()

    # Construct the sequence
    sequence = []
    if start_waypoint:
        sequence.append(start_waypoint)
    sequence.extend(intermediate_waypoints)
    if end_waypoint:
        sequence.append(end_waypoint)

    return sequence

# Apply the function to each group and concatenate the results
sequences = df_cph_Journeys.apply(create_sequence)

# Reset index to make it a regular DataFrame
sequences = sequences.reset_index(drop=True)


/var/folders/gh/zbst6d7n5434l6003rnm5nph0000gn/T/ipykernel_17654/1804335045.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sequences = df_cph_Journeys.apply(create_sequence)


In [9]:
sequences

0            [Nørreport St , Nørreport St., Helsingør St.]
1        [CPH Lufthavn , Lufthavnen St. (Metro), Nørrep...
2        [Lufthavnen St Metro , Hovedbanegården, Tivoli...
3        [Tivoli Hotel Kalvebod Brygge , Tivoli Hotel (...
4        [København H , Viby Sjælland St. (Søndergade),...
                               ...                        
16795           [Sjælør St , Ny Ellebjerg St., Sjælør St.]
16796    [Nørreport St , Nørreport St., Nørreport St., ...
16797    [Frederikssundsvej  Brønshøj Københavns Kommun...
16798                                      [Nørreport St.]
16799            [København H , København H, Ringsted St.]
Length: 16800, dtype: object

Testing with the first one: ['Nørreport St ', 'Nørreport St.', 'Helsingør St ']

In [10]:
df_cph_Journeys_list = df_All[(condition_1 | condition_2 | condition_3)].groupby('SJSearchJourney_Id').agg(list)
df_cph_Journeys_list.reset_index(inplace=True)
#Get the first group's SJSearchJourney_Id
first_group_id = df_cph_Journeys_list['SJSearchJourney_Id'].iloc[0]

#Filter the ungrouped DataFrame to include only the rows from the first group
first_group_rows = df_All[df_All['SJSearchJourney_Id'] == first_group_id]
first_group_rows

,JId,CreatedOn,SearchStart,SearchEnd,StartStop,EndStop,StartZone,Endzone,internalStartZones,internalValidZones,SJId,Id,Name,Latitude,Longitude,Type,SJSearchJourney_Id
90900,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",75d9ad38-0dc0-4864-bb6e-a978d8f352a7,8600646.0,Nørreport St.,55.683455,12.571801,REG,0009cc7f-0b72-49ae-92ba-486c59456bf8
90901,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",77dd5a4c-9e0c-4b51-8dae-bbb53feaf6e4,8600665.0,Nivå St.,55.933337,12.506575,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90902,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",2072f636-e799-4dba-850b-c725b4198e2f,8600661.0,Skodsborg St.,55.823534,12.571288,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90903,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",ce72eedb-2c4d-4682-a08e-dee25e33c30c,8600668.0,Snekkersten St.,56.009233,12.583882,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90904,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",71a36bd9-2540-40fb-8964-d0b73f5ae20a,8600663.0,Rungsted Kyst St.,55.882224,12.531475,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90905,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",ad991f45-99cf-45de-95bc-67cfb149cb8e,8600666.0,Humlebæk St.,55.963739,12.533408,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90906,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",241c9e86-a074-4774-b456-9439526af07f,8600655.0,Hellerup St.,55.731026,12.567657,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90907,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",6b98ca5d-45ac-4201-a85c-955c48dd6ed6,8600659.0,Klampenborg St.,55.777446,12.587730,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90908,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",10328d0e-3bce-4ccb-bdc4-0e0e8518606b,8600667.0,Espergærde St.,55.995803,12.549382,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8
90909,fabd60e9-a91c-4dcd-ad75-3e3aa20a32bc,2022-12-08 22.54.07.2887082,Nørreport St. (01),Helsingør St. (05),8600646.0,8600669.0,1001,1005,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013,1...",53431fb4-916b-4414-95d1-3132ee595d39,8600650.0,Østerport St.,55.692498,12.587784,waypoint,0009cc7f-0b72-49ae-92ba-486c59456bf8


Testing the second: ['CPH Lufthavn ', 'Lufthavnen St. (Metro)', 'Nørreport St.', 'Hillerød St ']

In [11]:

#Get the second group's SJSearchJourney_Id
second_group_id = df_cph_Journeys_list['SJSearchJourney_Id'].iloc[1]

#Filter the ungrouped DataFrame to include only the rows from the first group
second_group_rows = df_All[df_All['SJSearchJourney_Id'] == second_group_id]
second_group_rows

,JId,CreatedOn,SearchStart,SearchEnd,StartStop,EndStop,StartZone,Endzone,internalStartZones,internalValidZones,SJId,Id,Name,Latitude,Longitude,Type,SJSearchJourney_Id
234716,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",adb0bd46-403f-44cb-92d8-6570b213d6ec,8600677.0,Holte St.,55.807677,12.468820,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234717,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",9429e6f6-5ddc-4108-ad53-4fe86c800fba,8600655.0,Hellerup St.,55.731026,12.567657,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234718,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",9a328e51-f359-4b69-92b1-869cca609cd1,8600636.0,Sorgenfri St.,55.781257,12.483545,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234719,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",743dfba6-55a5-4806-8940-8c97113c8020,8600673.0,Gentofte St.,55.753508,12.541624,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234720,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",d7a99112-0dad-48d8-9171-8bf652977a5d,8600672.0,Bernstorffsvej St.,55.743053,12.557490,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234721,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",f12aa007-6f35-43f6-8b84-78687d68f2aa,8600678.0,Birkerød St.,55.840505,12.424018,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234722,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",124c2063-ae09-44d8-82e9-3da457d7839c,8603321.0,Amagerbro St. (Metro),55.663328,12.602930,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234723,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",7b976ac2-8175-46c3-ace5-3d3a0508bff2,8600674.0,Jægersborg St.,55.761625,12.521560,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234724,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",3e2a3072-5cae-4798-a6cb-48cd1f673c83,8600650.0,Østerport St.,55.692498,12.587784,waypoint,000c668a-e0de-4354-911a-317162ea3ebe
234725,3908d751-9933-428d-b7e2-acc4de7ec064,2022-12-02 17.43.18.6214314,CPH Lufthavn (04),Hillerød St. (09),8603328.0,8600683.0,1004,1009,1004,"1004,1003,1001,1002,1030,1041,1051,1061,1071,1...",b66cbb28-1919-475c-b8ab-2b6f608d4059,8600675.0,Lyngby St.,55.768088,12.503105,waypoint,000c668a-e0de-4354-911a-317162ea3ebe


Testing the second last: ['Nørreport St.']. Here we have one where SearchStart and SearchEnd are NaN. So we need to handle the startstop and endstop somehow. 

In [12]:
16798
#Get the second group's SJSearchJourney_Id
secondlast_group_id = df_cph_Journeys_list['SJSearchJourney_Id'].iloc[16798]

#Filter the ungrouped DataFrame to include only the rows from the first group
secondlast_group_rows = df_All[df_All['SJSearchJourney_Id'] == secondlast_group_id]
secondlast_group_rows

,JId,CreatedOn,SearchStart,SearchEnd,StartStop,EndStop,StartZone,Endzone,internalStartZones,internalValidZones,SJId,Id,Name,Latitude,Longitude,Type,SJSearchJourney_Id
19169,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",cd0c1d09-eba6-499f-956c-bdfcb2c4ab8e,NaN,Nørreport St.,55.683455,12.571801,waypoint,fffb52c2-f23c-4309-9e58-0fa514c44e07
19170,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",3ed33946-16e9-4fbb-8d2a-d08fef525489,NaN,til fods,55.683437,12.571576,WALK,fffb52c2-f23c-4309-9e58-0fa514c44e07
19171,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",dcb327e1-d5ad-4b1a-a409-d969ab769a7e,8600665.0,Nivå St.,55.933337,12.506575,waypoint,fffb52c2-f23c-4309-9e58-0fa514c44e07
19172,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",5c909983-bfda-402e-9038-9f27acb6f610,8600659.0,Klampenborg St.,55.777446,12.587730,waypoint,fffb52c2-f23c-4309-9e58-0fa514c44e07
19173,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",467c5ec6-e29e-4144-ad1f-a76e0770263a,8600661.0,Skodsborg St.,55.823534,12.571288,waypoint,fffb52c2-f23c-4309-9e58-0fa514c44e07
19174,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",b83531dd-7b47-40b4-8c0e-a197a058bd1c,8600646.0,Nørreport St.,55.683455,12.571801,REG,fffb52c2-f23c-4309-9e58-0fa514c44e07
19175,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",e6586f17-a15c-47ee-9e7d-14e9b64710b1,8600664.0,Kokkedal St.,55.903484,12.502054,waypoint,fffb52c2-f23c-4309-9e58-0fa514c44e07
19176,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",4433a59c-dbbc-4c37-9ca4-0d0891d10839,NaN,Humlebæk St.,55.963739,12.533408,WALK,fffb52c2-f23c-4309-9e58-0fa514c44e07
19177,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",c589669b-a706-4f2f-aa2e-35c4a31e5f83,NaN,til fods,55.969411,12.540240,waypoint,fffb52c2-f23c-4309-9e58-0fa514c44e07
19178,35217071-ffb4-4287-a2f5-44190a00a5b7,2022-12-11 12.46.29.3149455,NaN,NaN,8600646.0,8600666.0,1001,1013,1001,"1001,1002,1030,1040,1050,1060,1070,1080,1013",871bdf56-ba3d-4223-9e3f-301b35d9366b,8600666.0,Humlebæk St.,55.963739,12.533408,waypoint,fffb52c2-f23c-4309-9e58-0fa514c44e07


For each sequence we wish to sort the sequence so the the journey is in the correct order aka that all waypoints appear in correct order. Doing so by sorting based on euclidian (diagional) distance from thee SearchStart. 

Updating sequences to contain latitude and longitude as well to be able to sort them according to their distance to the starting stop:
## I have run into an issue, how do we get the coordinates of the searchStart and SearchEnd, when they are not guarenteed to be in the list of waypoints?

In [55]:
from difflib import SequenceMatcher

def create_sequence(group):
    # Remove non-alphanumeric characters from the SearchStart and SearchEnd columns
    group['SearchStart'] = group['SearchStart'].apply(remove_non_alphanumeric)
    group['SearchEnd'] = group['SearchEnd'].apply(remove_non_alphanumeric)

    # Find the starting waypoint
    start_waypoint = group['SearchStart'].iloc[0] if not pd.isnull(group['SearchStart'].iloc[0]) else None
    
    # In this collection, we might not be able to find the last stop - the destination of a journey.
    # if SearchEnd is not null, look through the list of names for the most similar one
    if not pd.isna(group['SearchEnd'].iloc[0]):
        names = group['Name'].values.tolist()
        end = group['SearchEnd'].iloc[0]
        
        # Find the station in names that is most similar to the searchEnd station 
        # To "take" it out of the list
        currhigh = ('', 0)
        for name in names:
            s = SequenceMatcher(a=end, b=name).ratio()
            if s > currhigh[1]:
                currhigh = (name, s)
        end_waypoint = currhigh[0] 
    else:
        end_waypoint = None

    # Find the intermediate waypoints and their latitude and longitude
    intermediate_waypoints = group.loc[(group['Type'] != 'WALK') & (group['Type'] != 'waypoint') & (group['Name'] != end_waypoint)]
    #intermediate_waypoints = intermediate_waypoints[['Name', 'Latitude', 'Longitude']].values.tolist()
    intermediate_waypoints = intermediate_waypoints[['Name', 'Latitude', 'Longitude']].to_dict('records')


    # Construct the sequence adding the latitude and longitude to each member of the sequence as well. 
    sequence = []
    if start_waypoint:
        sequence.append({'Name': start_waypoint, 'Latitude': group['Latitude'].iloc[0], 'Longitude': group['Longitude'].iloc[0]})
    sequence.extend(intermediate_waypoints)
    if end_waypoint:
        sequence.append({'Name': end_waypoint, 'Latitude': group['Latitude'].iloc[0], 'Longitude': group['Longitude'].iloc[0]})

    return sequence

# Apply the function to each group and concatenate the results
sequencesWithCoords = df_cph_Journeys.apply(create_sequence)

# Reset index to make it a regular DataFrame
sequencesWithCoords = sequencesWithCoords.reset_index(drop=True)


/var/folders/gh/zbst6d7n5434l6003rnm5nph0000gn/T/ipykernel_17654/4038280693.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sequencesWithCoords = df_cph_Journeys.apply(create_sequence)


Testing:

In [77]:
# Access the first list in the sequences Series
first_sequence = sequencesWithCoords.iloc[25]

# Print the names and associated latitudes and longitudes for each element in the first list
first_sequence


[{'Name': 'Lundtoftegade   København N Københavns Kommune',
  'Latitude': 55.699528,
  'Longitude': 12.469387},
 {'Name': 'Nørrebro St.', 'Latitude': 55.700849, 'Longitude': 12.537804},
 {'Name': 'Flintholm St.', 'Latitude': 55.685432, 'Longitude': 12.498656},
 {'Name': 'Borups Plads', 'Latitude': 55.691482, 'Longitude': 12.538873},
 {'Name': 'Fuglebakken St.', 'Latitude': 55.699528, 'Longitude': 12.469387}]

In [61]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

#Each element of a sequence is a dictionary that contains Name, lat and long of the given station
def sort_waypoints(waypoints, search_start_lat, search_start_lon):
    sorted_waypoints = sorted(waypoints, key=lambda waypoint: euclidean_distance(waypoint['Latitude'], waypoint['Longitude'], search_start_lat, search_start_lon))
    return sorted_waypoints

# Assuming sequenceWithCoords is a list of sequences where each sequence contains station names and coordinates
sorted_sequences = []

# Iterate over each sequence in sequenceWithCoords
for sequence in sequencesWithCoords:
    # Extract the search start latitude and longitude from the first waypoint in the sequence

    if(sequence == sequencesWithCoords[2]):
        search_start_lat = sequence[0]['Latitude']
        search_start_lon = sequence[0]['Longitude']
        
        # Sort the waypoints in the sequence based on the search start coordinates
        sorted_sequence = sort_waypoints(sequence, search_start_lat, search_start_lon)
        
        # Append the sorted sequence to the list of sorted sequences
        sorted_sequences.append(sorted_sequence)


In [62]:
sorted_sequences

[[{'Name': 'Lufthavnen St Metro ',
   'Latitude': 55.628432,
   'Longitude': 12.578776},
  {'Name': 'CPH Lufthavn', 'Latitude': 55.628432, 'Longitude': 12.578776},
  {'Name': 'Hovedbanegården, Tivoli (Bernstorffsgade)',
   'Latitude': 55.672839,
   'Longitude': 12.566192},
  {'Name': 'Hovedbanegården, Tivoli (Bernstorffsgade)',
   'Latitude': 55.672839,
   'Longitude': 12.566192}]]

### Stuck in sorting the sequences, because of the missing correct latitude and longitude of searchStart and searchEnd,. Why is Fuglebakken there in journey 25?